# Moomoo version 1.0

## Initialize

In [3]:
import requests
#import pdfplumber
import os, fnmatch
import re
import pandas as pd
from pypdf import PdfReader

## List of files

In [9]:
# find a file to extract
path = '../extract_pdf/'
arr = fnmatch.filter(os.listdir(path), '*.pdf')

# sorted(arr)
# print(len(arr))
print(arr)
print(len(arr))

['1017286201753723-6-20240531-1717486725753.pdf', '1017286201753723-6-20240612-1718891805080.pdf', '1017286201753723-6-20240614-1718891814548.pdf']
3


## Read PDF files and convert into text

In [5]:
def read_pdf(pdf_path):
    reader = PdfReader(pdf_path)

    # Print the number of pages in the PDF
    #print(f"There are {len(reader.pages)} Pages")

    # Get the first page (index 0) 
    page = reader.pages[0]
    # Use extract_text() to get the text of the page
    #print(page.extract_text())
    text =''

    # Go through every page and get the text
    for i in range(len(reader.pages)):
      page = reader.pages[i]
      text = text + page.extract_text()
      #print(text)
    
    return text

In [13]:
text = read_pdf(path+arr[0])
print(text)

Account Information
Account Information
Name
MOHAMAD KHAIRUL AKMAL BIN AMRAN
MOHAMAD KHAIRUL AKMAL BIN AMRAN
Customer Type
Client
Client
Account Number
1017286201753723
1017286201753723
Account Type
Margin(Next Financing Renewal Date:2024/06/22)
Margin(Next Financing Renewal Date:2024/06/22)
Address
F-11-08 PANGSAPURI PPAM PUDINA, JALAN KASTURI P17, PRESINT 17,62100,Putrajaya,WILAYAH PERSEKUTUAN,Malaysia
F-11-08 PANGSAPURI PPAM PUDINA, JALAN KASTURI P17, PRESINT 17,62100,Putrajaya,WILAYAH PERSEKUTUAN,Malaysia
Net Asset Value: 
Net Asset Value: 
4,876.96
4,876.96
Base Currency: MYR
Assets in Transit
0.00
0.00
Portfolio Value
1,492.00
1,492.00
Cash Balance
3,384.96
3,384.96
Changes in Net Asset Value
Changes in Net Asset Value
Starting Net Asset Value 20240401
Equal to(MYR)
4,947.50
4,947.50
 
MYR
4,947.50
4,947.50
exchange rate : 
1.0000
 
USD
0.00
0.00
exchange rate : 
-
Ending Net Asset Value 20240402
Equal to(MYR)
4,876.96
4,876.96
 
MYR
4,876.96
4,876.96
exchange rate : 
1.0000
 
US

## Select extract section

In [6]:
def extract_section(text, start_word, end_word):
    # Create a regex pattern to capture the text between start_word and end_word
    pattern = re.compile(rf"{re.escape(start_word)}(.*?){re.escape(end_word)}", re.DOTALL)
    match = pattern.search(text)
    if match:
        return match.group(0)  # Return the entire match (including start and end words)
    else:
        return None

In [7]:
# Extract text from "Direction" until "Settlement Date"
start_word = "Trades - Securities"
end_word = "Changes in Cash"
extracted_text = extract_section(text, start_word, end_word)

print(extracted_text)

NameError: name 'text' is not defined

## Regex pattern match to extract the required data

In [8]:
def match_pattern(text,option=True):

    if option:
        # Regex pattern to match the required fields
        pattern = re.compile(
            r'(Buy to Open|Sell to Close)\s+(\w+)\s+(\d{4})\s+[A-Z]+\s+[A-Z]+\s+(\d{4})\/(\d{2})\/(\d{2})\s+\d{2}:\d{2}:\d{2}\s+(\d+\.\d{4})\s+(\d+)'
        )
    else:
        # Regex pattern to match the required fields
        pattern = re.compile(
            #r'Net Transaction Amount:\s*(-?\d{1,3}(?:,\d{3})*\.\d{2})\s+-?\d{1,3}(?:,\d{3})*\.\d{2}\s+Commission:\s*(\d+\.\d{2})\s+\d+\.\d{2}\s+Platform Fees:\s*(\d+\.\d{2})\s+\d+\.\d{2}\s+Clearing Fee:\s*(\d+\.\d{2})\s+\d+\.\d{2}\s+Stamp Duty:\s*(\d+\.\d{2})\s+\d+\.\d{2}\s+'
            r'Net Transaction Amount:\s*(-?\d{1,3}(?:,\d{3})*\.\d{2})\s+-?\d{1,3}(?:,\d{3})*\.\d{2}\s+Commission:\s*(\d+\.\d{2})\s+\d+\.\d{2}\s+Platform Fees:\s*(\d+\.\d{2})\s+\d+\.\d{2}\s+Clearing Fee:\s*(\d+\.\d{2})\s+\d+\.\d{2}\s+Stamp Duty:\s*(\d+\.\d{2})\s+\d+\.\d{2}\s+Settlement Date:\s+\d{4}\/\d{2}\/\d{2}\s+\d{4}\/\d{2}\/\d{2}\s+Reference No:\s*([\w\/]+)'
        )

    # Search for the pattern in the text
    match = pattern.findall(text)

    # If a match is found, extract the groups
    if match:
        #extracted_data = match.groups()
        #print(extracted_data)
        return match
        
    else:
        return "No match found."
        
    


In [ ]:
print(match_pattern(text, option=True))

In [ ]:
print(match_pattern(text, option=False))

## Handle data to standard format

In [9]:
def final_data(text):
    stock = match_pattern(text, option=True)
    fee = match_pattern(text, option=False)
    
    #convert to dataframe
    df_stock = pd.DataFrame(stock)
    df_stock.rename(columns ={0 : 'position', 1 : 'stock', 2 : 'code', 3 : 'year', 4 : 'month', 5 : 'day', 6 : 'price', 7 : 'unit'}, inplace = True)
    df_stock['date'] = df_stock['day'] + '/' + df_stock['month'] + '/' + df_stock['year']
    df_stock['unit'] = df_stock['unit'].astype(int)
    df_stock['price'] = df_stock['price'].astype(float)
    df_stock['subtotal'] = df_stock['price'] * df_stock['unit']

    #df_stock
    
    df_fee = pd.DataFrame(fee)
    df_fee.rename(columns ={0 : 'total', 1 : 'commision', 2 : 'brk_fee', 3 : 'clear_fee', 4 : 'stamp', 5 : 'ref'}, inplace = True)
    df_fee['total'] = df_fee['total'].str.replace(',', '').astype(float)
    df_fee['total'] = df_fee['total'].abs()
    
    result = pd.concat([df_stock, df_fee], axis=1)
    new_order = ['date', 'day', 'month', 'year', 'ref', 'code', 'stock', 'price', 'unit','subtotal', 'brk_fee', 'stamp', 'clear_fee', 'commision','total', 'position']
    result = result[new_order]
    
    return result
    

## Final result (manual)

In [31]:
text = read_pdf(path+arr[5])
# Extract text from "Direction" until "Settlement Date"
#start_word = "Trades - Securities"
#end_word = "Changes in Cash"
#extracted_text = extract_section(text, start_word, end_word)
stock = match_pattern(text, option=True)
fee = match_pattern(text, option=False)
print(stock)
print(fee)

[('Buy to Open', 'SYNERGY', '0279', '2024', '04', '24', '1.3100', '200')]
[('-266.08', '0.00', '3.00', '0.08', '1.00', 'v3_20240424_1093797161_j/KjNgXpD3YA')]


In [ ]:
df_stock = pd.DataFrame(stock)
df_stock.rename(columns ={0 : 'position', 1 : 'stock', 2 : 'code', 3 : 'year', 4 : 'month', 5 : 'day', 6 : 'price', 7 : 'unit'}, inplace = True)

df_stock['date'] = df_stock['day'] + '/' + df_stock['month'] + '/' + df_stock['year']

df_stock['unit'] = df_stock['unit'].astype(int)
df_stock['price'] = df_stock['price'].astype(float)
df_stock['subtotal'] = df_stock['price'] * df_stock['unit']

df_stock


In [ ]:
df_fee = pd.DataFrame(fee)
df_fee.rename(columns ={0 : 'total', 1 : 'commision', 2 : 'brk_fee', 3 : 'clear_fee', 4 : 'stamp'}, inplace = True)
df_fee['total'] = df_fee['total'].str.replace(',', '').astype(float)
df_fee['total'] = df_fee['total'].abs()
df_fee

In [ ]:
result = pd.concat([df_stock, df_fee], axis=1)
new_order = ['day', 'month', 'year', 'code', 'stock', 'price', 'unit', 'brk_fee', 'stamp', 'clear_fee', 'total', 'position']
result = result[new_order]
result

## Final Result (Auto)

In [10]:
# find a file to extract
path = '../extract_pdf/'
arr = fnmatch.filter(os.listdir(path), '*.pdf')
len(arr)

3

In [11]:
text = read_pdf(path+arr[0])
final = final_data(text)
#result=result.append(final, ignore_index=True)
final

,date,day,month,year,ref,code,stock,price,unit,subtotal,brk_fee,stamp,clear_fee,commision,total,position
0,31/05/2024,31,05,2024,560dfe0678ee42fb8b98d15943fa474a,0089,TEXCYCL,1.38,400,552.0,3.00,1.00,0.17,0.00,556.17,Buy to Open


In [ ]:
arr[15]

## Final Result (iteration)

In [12]:
# find a file to extract
path = '../extract_pdf/'
arr = fnmatch.filter(os.listdir(path), '*.pdf')

for x in range(len(arr)):
    text = read_pdf(path+arr[x])
    df=final_data(text)
    df.to_csv(path + 'result.csv', mode='a', index=False, header=False)
    df = df[['date','stock','price','unit','total','position']]
    df.to_csv(path + 'key.csv', mode='a', index=False, header=False)
    
#result.to_csv(path + 'result.csv')
    
  

In [13]:
df = df[['date','stock','price','unit','total','position']]
df

,date,stock,price,unit,total,position
0,14/06/2024,GAMUDA,6.71,100,675.21,Buy to Open


In [14]:
result.to_csv('result.csv')

NameError: name 'result' is not defined

## Move file

In [16]:
import os

# find a file to extract
path = '../extract_pdf/'
arr = fnmatch.filter(os.listdir(path), '*.pdf')

arr


['1017286201753723-6-20240531-1717486725753.pdf',
 '1017286201753723-6-20240612-1718891805080.pdf',
 '1017286201753723-6-20240614-1718891814548.pdf']

In [17]:
# Define the source path and the destination directory
source_path = path + arr[0]
destination_directory = '../Moomoo_pdf/'

# Construct the full destination path
destination_path = os.path.join(destination_directory, os.path.basename(source_path))

# Move the file
os.rename(source_path, destination_path)